#  Auto Sales Data Analysis And Evaluation

 # Main Question:

Why do some countries have higher auto sales than others, and what factors (e.g., price, quantity, product line, and customer characteristics) contribute to these differences?

# Sub Questions:
Does quantity ordered significantly relate to total sales across countries or customer types? Can we predict whether a product will sell based on features like price, product line,and deal size? Can we use the number of days that have passed since the last order for each customer to analyze customer purchasing patterns?

In [26]:
import os

import numpy as np
import pandas as pd

import plotly.express as px

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")


In [27]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/auto-sales-data/Auto Sales data.csv


In [30]:
df = pd.read_csv("/kaggle/input/auto-sales-data/Auto Sales data.csv", index_col=0)

In [31]:
df.columns

Index(['QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER', 'SALES', 'ORDERDATE',
       'DAYS_SINCE_LASTORDER', 'STATUS', 'PRODUCTLINE', 'MSRP', 'PRODUCTCODE',
       'CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'CITY', 'POSTALCODE',
       'COUNTRY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME', 'DEALSIZE'],
      dtype='object')

In [32]:
df.head

<bound method NDFrame.head of              QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES   ORDERDATE  \
ORDERNUMBER                                                                     
10107                     30      95.70                2  2871.00  24/02/2018   
10121                     34      81.35                5  2765.90  07/05/2018   
10134                     41      94.74                2  3884.34  01/07/2018   
10145                     45      83.26                6  3746.70  25/08/2018   
10168                     36      96.66                1  3479.76  28/10/2018   
...                      ...        ...              ...      ...         ...   
10350                     20     112.22               15  2244.40  02/12/2019   
10373                     29     137.19                1  3978.51  31/01/2020   
10386                     43     125.99                4  5417.57  01/03/2020   
10397                     34      62.24                1  2116.16  28/03/2020  

In [33]:
df.info

<bound method DataFrame.info of              QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES   ORDERDATE  \
ORDERNUMBER                                                                     
10107                     30      95.70                2  2871.00  24/02/2018   
10121                     34      81.35                5  2765.90  07/05/2018   
10134                     41      94.74                2  3884.34  01/07/2018   
10145                     45      83.26                6  3746.70  25/08/2018   
10168                     36      96.66                1  3479.76  28/10/2018   
...                      ...        ...              ...      ...         ...   
10350                     20     112.22               15  2244.40  02/12/2019   
10373                     29     137.19                1  3978.51  31/01/2020   
10386                     43     125.99                4  5417.57  01/03/2020   
10397                     34      62.24                1  2116.16  28/03/2020

In [34]:
df.describe

<bound method NDFrame.describe of              QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES   ORDERDATE  \
ORDERNUMBER                                                                     
10107                     30      95.70                2  2871.00  24/02/2018   
10121                     34      81.35                5  2765.90  07/05/2018   
10134                     41      94.74                2  3884.34  01/07/2018   
10145                     45      83.26                6  3746.70  25/08/2018   
10168                     36      96.66                1  3479.76  28/10/2018   
...                      ...        ...              ...      ...         ...   
10350                     20     112.22               15  2244.40  02/12/2019   
10373                     29     137.19                1  3978.51  31/01/2020   
10386                     43     125.99                4  5417.57  01/03/2020   
10397                     34      62.24                1  2116.16  28/03/20

In [35]:
df.isnull().sum()

QUANTITYORDERED         0
PRICEEACH               0
ORDERLINENUMBER         0
SALES                   0
ORDERDATE               0
DAYS_SINCE_LASTORDER    0
STATUS                  0
PRODUCTLINE             0
MSRP                    0
PRODUCTCODE             0
CUSTOMERNAME            0
PHONE                   0
ADDRESSLINE1            0
CITY                    0
POSTALCODE              0
COUNTRY                 0
CONTACTLASTNAME         0
CONTACTFIRSTNAME        0
DEALSIZE                0
dtype: int64

In [36]:
sales_country = df.groupby("COUNTRY")["SALES"].sum().reset_index()
fig1 = px.bar(df,
             x="COUNTRY",
              y="SALES",
              color="CITY",
              title="Total Sales per Country",
             )
fig1.update_layout(title_x=0.5)
fig1.show()

In [ ]:
#The plot reveals that the United States leads in total sales, followed by Spain and France. According to the dataset, a significantly higher volume of goods is sold to customers in the U.S. compared to other countries.

In [37]:
fig2 = px.scatter(df,
                x="QUANTITYORDERED",
                y="SALES",
                color="PRODUCTLINE",
                title="QUANTITY ORDERED vs SALES",
                )
fig2.update_layout(title_x=0.5)
fig2.show()

In [38]:
#The scatter plot reveals that the majority of sales occur when the quantity ordered is between 20 and 40 units, with corresponding sales values mostly between $2,000 and $10,000. This suggests a typical order volume and sales range for most products. Additionally, by observing the color-coded clusters, I can identify which product lines (e.g.,Vintage Cars, Classic Cars,Motorcycles, etc.) are more frequently sold within this high-activity range.

In [39]:
filtered_df = df[(df['QUANTITYORDERED'] >= 20) & (df['QUANTITYORDERED'] <= 40)]
product_counts = filtered_df['PRODUCTLINE'].value_counts().reset_index()
product_counts.columns = ['PRODUCTLINE', 'OrderCount']

print(product_counts)

        PRODUCTLINE  OrderCount
0      Classic Cars         636
1      Vintage Cars         388
2       Motorcycles         211
3            Planes         204
4  Trucks and Buses         180
5             Ships         163
6            Trains          49


In [40]:
fig3 = px.bar(product_counts, x='PRODUCTLINE', y='OrderCount',
             title='Most Sold Product Lines (Quantity Ordered Between 20 and 40)',
             labels={'OrderCount': 'Number of Orders'})
fig3.update_layout(title_x=0.5)
fig3.show()

In [41]:
#Most sold productline is CLASSIC CARS

In [42]:
fig4 = px.box(df, x='DEALSIZE', y='PRICEEACH', title='Price Distribution per Deal Size')
fig4.update_layout(title_x=0.5)
fig4.show()

In [43]:
#The plot shows that large deals are typically made for products priced between 100 and 250. This suggests that higher-priced items are more common in large volume purchases, though it doesn’t necessarily mean they are sold more frequently.

In [44]:
country_summary = df.groupby('COUNTRY').agg({
    'SALES': 'sum',
    'QUANTITYORDERED': 'mean',
    'PRICEEACH': 'mean',
    'DEALSIZE': pd.Series.mode,
    'PRODUCTLINE': pd.Series.mode  
}).reset_index()

print(country_summary)

        COUNTRY       SALES  QUANTITYORDERED   PRICEEACH DEALSIZE  \
0     Australia   630623.10        33.762162  101.383514    Small   
1       Austria   202062.53        35.890909  100.493818   Medium   
2       Belgium   108412.62        32.545455   98.755152    Small   
3        Canada   224078.56        32.757143   98.064286    Small   
4       Denmark   245637.15        34.873016  109.850794   Medium   
5       Finland   329581.91        34.695652  103.570870   Medium   
6        France  1110916.52        35.318471   99.102166   Medium   
7       Germany   220472.09        34.645161  103.228710   Medium   
8       Ireland    57756.43        30.625000  112.870000    Small   
9         Italy   374674.31        33.389381  100.260442    Small   
10        Japan   188167.81        35.423077  100.694808    Small   
11       Norway   307463.70        33.435294  107.275882   Medium   
12  Philippines    94015.73        36.961538   96.080000   Medium   
13    Singapore   288488.41       

 # USA leads in total sales, which appears to be driven by a combination of:
    •	Higher average order quantity (36 units/order)
	•	Higher average price per unit (~115)
	•	Predominantly large deal sizes
	•	 Strong performance in the “Classic Cars” product line

In [45]:
fig5= px.histogram(df, x="DAYS_SINCE_LASTORDER", nbins=20,
                   title="Distribution of Days Since Last Order",
                  color="DEALSIZE")
fig5.update_layout(title_x=0.5)
fig5.show()

In [46]:
fig6 = px.scatter(df, 
                 x="DAYS_SINCE_LASTORDER", 
                 y="SALES", 
                 trendline="ols", 
                 title="Relationship Between Days Since Last Order and Sales")
fig6.update_layout(title_x=0.5)
fig6.show()

In [47]:
#As the number of days since the last order increases, sales tend to decrease.

In [48]:
print(df[['DAYS_SINCE_LASTORDER', 'SALES']].corr())

                      DAYS_SINCE_LASTORDER     SALES
DAYS_SINCE_LASTORDER              1.000000 -0.334274
SALES                            -0.334274  1.000000


In [49]:
#Moderate Negative Correlation observed,which means the longer it’s been since a customer placed an order, the lower the sales tend to be.

In [50]:
threshold = df['SALES'].median()
df['HIGH_SALES'] = df['SALES'].apply(lambda x: 1 if x > threshold else 0)

In [51]:
df_encoded = pd.get_dummies(df[['DEALSIZE', 'PRODUCTLINE']], drop_first=True)

X = pd.concat([df[['PRICEEACH', 'QUANTITYORDERED']], df_encoded], axis=1)
y = df['HIGH_SALES']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       282
           1       0.95      0.99      0.97       268

    accuracy                           0.97       550
   macro avg       0.97      0.97      0.97       550
weighted avg       0.97      0.97      0.97       550



In [53]:
sample_data = pd.DataFrame([[
    95, 0,  14, 0, 0, 0, 0, 0, 10, 0    
]], columns=X.columns)

#example Prediction
sample_prediction = model.predict(sample_data)
print("Prediction:", "High Sales" if sample_prediction[0] == 1 else "Low Sales")

Prediction: Low Sales


The logistic regression model performs very well, with 97% accuracy in predicting whether a product will have high or low sales. It shows a strong balance between correctly identifying both categories.

Key factors like price, quantity, deal size, and product line help the model make accurate predictions. This model can support better decisions in pricing, marketing, and inventory planning.

Overall, it’s a reliable and useful tool for sales classification.